<a href="https://colab.research.google.com/github/alesaccoia/DDM_2425/blob/main/AI_003_Retrieval_ed_Embeddings%2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval ed Embeddings

In [1]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Knowledge base con frasi strutturate che menzionano sempre la regione
knowledge_base = [
    # Toscana
    "La bistecca alla fiorentina è un piatto tipico della regione Toscana",
    "La ribollita è una zuppa tradizionale della regione Toscana",
    "Il cacciucco è una zuppa di pesce originaria della città di Livorno, in Toscana",
    "I cantucci sono biscotti alle mandorle tradizionali della regione Toscana",
    "Il pecorino di Pienza è un formaggio DOP prodotto in Toscana",
    "Il Brunello è un vino prodotto a Montalcino, in Toscana",

    # Sicilia
    "Gli arancini sono palle di riso ripiene tipiche della regione Sicilia",
    "La pasta alla Norma è un piatto tradizionale della città di Catania, in Sicilia",
    "I cannoli ripieni di ricotta sono dolci tipici della regione Sicilia",
    "La caponata di melanzane è un contorno tradizionale della regione Sicilia",
    "Il pistacchio di Bronte è coltivato in Sicilia ed è un prodotto DOP",
    "La cassata è un dolce tradizionale della regione Sicilia",

    # Emilia-Romagna
    "I tortellini in brodo sono il piatto simbolo della città di Bologna, in Emilia-Romagna",
    "Il Parmigiano Reggiano DOP è prodotto in Emilia-Romagna",
    "La mortadella IGP è un salume tipico della città di Bologna, in Emilia-Romagna",
    "I cappellacci di zucca sono un primo piatto tipico di Ferrara, in Emilia-Romagna",
    "L'aceto balsamico tradizionale DOP è prodotto a Modena, in Emilia-Romagna",
    "La piadina è una focaccia tipica della Romagna, in Emilia-Romagna",

    # Campania
    "La pizza Margherita è stata inventata a Napoli, in Campania",
    "La mozzarella di bufala DOP è prodotta in Campania",
    "Il babà al rum è un dolce tipico di Napoli, in Campania",
    "La sfogliatella riccia è un dolce tradizionale di Napoli, in Campania",
    "La pasta alla Genovese è un piatto tipico di Napoli, in Campania",
    "Il limoncello è un liquore prodotto nella costiera amalfitana, in Campania",

    # Veneto
    "Il baccalà alla vicentina è un piatto tipico di Vicenza, in Veneto",
    "Il risotto all'Amarone è un piatto della Valpolicella, in Veneto",
    "Il tiramisù è un dolce originario della città di Treviso, in Veneto",
    "I bigoli in salsa sono una pasta tipica di Venezia, in Veneto",
    "Il fegato alla veneziana è un piatto tradizionale di Venezia, in Veneto",
    "Il prosecco DOC è un vino prodotto in Veneto",

    # Piemonte
    "Il tartufo bianco d'Alba è un prodotto tipico del Piemonte",
    "I tajarin sono tagliolini all'uovo tipici del Piemonte",
    "Il brasato al Barolo è un secondo piatto tradizionale del Piemonte",
    "Il gianduiotto è un cioccolatino nato a Torino, in Piemonte",
    "La bagna càuda è una salsa tipica della tradizione piemontese",
    "Il Barolo è un vino DOCG prodotto in Piemonte"
]



/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:

class DenseRetrieval:
    def __init__(self, sentences):
        print("Inizializzazione del sistema di retrieval...")
        self.sentences = sentences
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        # Pre-calcola gli embedding per tutte le frasi
        self.embeddings = self.model.encode(sentences)
        print(f"Sistema inizializzato con {len(sentences)} frasi nella knowledge base")

    def search(self, query, k=3):
        """
        Cerca le k frasi più rilevanti per la query
        """
        # Calcola l'embedding della query
        query_embedding = self.model.encode([query])

        # Calcola similarità del coseno
        similarities = cosine_similarity(query_embedding, self.embeddings)[0]

        # Trova i k risultati migliori
        top_k_indices = similarities.argsort()[-k:][::-1]

        results = []
        for idx in top_k_indices:
            results.append({
                'text': self.sentences[idx],
                'similarity': similarities[idx]
            })

        return results


In [4]:
def demo_ricerca(retriever, query, k=3):
    """Funzione dimostrativa per testare il retrieval"""
    print(f"\nQuery: '{query}'")
    print("-" * 60)

    results = retriever.search(query, k=k)

    for i, result in enumerate(results, 1):
        print(f"{i}. [Score: {result['similarity']:.3f}] {result['text']}")



In [5]:
retriever = DenseRetrieval(knowledge_base)

# Esempi di query per testare il sistema
queries = [
    "Cosa si mangia a Napoli?",
    "Quali sono i prodotti tipici del Piemonte?",
    "Dimmi dei piatti con il pesce",
    "Quali sono i dolci più famosi?",
    "Cosa producono a Modena?"
]

# Testa ogni query
for query in queries:
    demo_ricerca(retriever, query, k=3)

Inizializzazione del sistema di retrieval...
Sistema inizializzato con 36 frasi nella knowledge base

Query: 'Cosa si mangia a Napoli?'
------------------------------------------------------------
1. [Score: 0.691] La pasta alla Genovese è un piatto tipico di Napoli, in Campania
2. [Score: 0.687] La sfogliatella riccia è un dolce tradizionale di Napoli, in Campania
3. [Score: 0.664] La pizza Margherita è stata inventata a Napoli, in Campania

Query: 'Quali sono i prodotti tipici del Piemonte?'
------------------------------------------------------------
1. [Score: 0.788] Il tartufo bianco d'Alba è un prodotto tipico del Piemonte
2. [Score: 0.745] I tajarin sono tagliolini all'uovo tipici del Piemonte
3. [Score: 0.691] Il Barolo è un vino DOCG prodotto in Piemonte

Query: 'Dimmi dei piatti con il pesce'
------------------------------------------------------------
1. [Score: 0.624] Il pecorino di Pienza è un formaggio DOP prodotto in Toscana
2. [Score: 0.623] La pasta alla Genovese è un 

Analizziamo i risultati:

Le query con riferimenti geografici espliciti funzionano molto bene:

* "Cosa si mangia a Napoli?" → trova correttamente tre piatti napoletani
* "Quali sono i prodotti tipici del Piemonte?" → trova correttamente tre specialità piemontesi

Questo succede perché il modello di embedding è molto bravo a catturare le relazioni tra nomi di luoghi e il loro contesto: l'associazione "Napoli-Campania" o "Piemonte-Alba" è fortemente presente nei dati di training.

Le query generiche o con concetti semantici più astratti funzionano male:
* "Dimmi dei piatti con il pesce" → non trova il cacciucco (che è una zuppa di pesce!) ma trova erroneamente pecorino, pasta alla genovese e vino
* "Quali sono i dolci più famosi?" → non trova né cannoli, né babà, né tiramisù, ma trova erroneamente paste e pistacchio
* "Cosa producono a Modena?" → trova correttamente l'aceto balsamico ma poi suggerisce prodotti di altre regioni

Questo accade per diverse ragioni:

* Il modello non ha una vera comprensione delle categorie (non "sa" davvero cosa è un dolce o cosa contiene pesce)
* Non è in grado di fare inferenze logiche (non "capisce" che una zuppa di pesce è un piatto di pesce)
* Si basa principalmente sulla similarità delle parole e del contesto in cui appaiono nei dati di training

Per migliorare questi risultati potremmo:

* Strutturare meglio le frasi includendo categorie esplicite: "Il cacciucco è un piatto a base di pesce tipico..."
* Usare un approccio ibrido con keyword matching per termini specifici come "dolce", "pesce", etc.
Aggiungere metadati o tag alle frasi per permettere filtri più precisi